In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
tf.constant([[1., 2., 3.], [4., 5., 6.]])  # metrix

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

In [6]:
t.shape

TensorShape([2, 3])

In [7]:
t.dtype

tf.float32

In [8]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [9]:
t[:, 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [10]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [11]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [12]:
K = keras.backend

In [13]:
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

## 텐서와 넘파이

In [14]:
a = np.array([2., 4., 5.])

In [15]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [16]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [17]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [18]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## 타입 변환

In [19]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)    # cast = 형변환

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

## 변수

In [20]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [21]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [22]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [23]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [24]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100, 200])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

## 사용자 정의모델과 훈련 알고리즘

In [25]:
housing = fetch_california_housing()
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state=42)

x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [26]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss =tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [27]:
input_shape = x_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [28]:
model.compile(loss = huber_fn, optimizer ='nadam')

In [29]:
model.fit(x_train_scaled, y_train, epochs= 10, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 2s 186us/sample - loss: 0.5984 - val_loss: 0.2900
Epoch 2/10
11610/11610 [==============================] - 1s 48us/sample - loss: 0.2067 - val_loss: 0.2281
Epoch 3/10
11610/11610 [==============================] - 1s 47us/sample - loss: 0.1969 - val_loss: 0.1816
Epoch 4/10
11610/11610 [==============================] - 1s 47us/sample - loss: 0.1915 - val_loss: 0.1883
Epoch 5/10
11610/11610 [==============================] - 1s 47us/sample - loss: 0.1891 - val_loss: 0.1810
Epoch 6/10
11610/11610 [==============================] - 1s 47us/sample - loss: 0.1868 - val_loss: 0.1766
Epoch 7/10
11610/11610 [==============================] - 1s 52us/sample - loss: 0.1848 - val_loss: 0.1811
Epoch 8/10
11610/11610 [==============================] - 1s 57us/sample - loss: 0.1831 - val_loss: 0.1853
Epoch 9/10
11610/11610 [==============================] - 1s 50us/sample - loss: 0.1815 - val_

In [30]:
model.save('models\my_model_with_a_custom_loo.h5')

In [31]:
model = keras.models.load_model('models\my_model_with_a_custom_loo.h5', custom_objects={'huber_fn':huber_fn})

In [32]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < 1
        squared_loss =tf.square(error) / 2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss = create_huber(2.0), optimizer = 'nadam')

In [33]:
model.fit(x_train_scaled, y_train, epochs= 10, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 1s 78us/sample - loss: 0.1785 - val_loss: 0.1661
Epoch 2/10
11610/11610 [==============================] - 1s 66us/sample - loss: 0.1767 - val_loss: 0.1642
Epoch 3/10
11610/11610 [==============================] - 1s 46us/sample - loss: 0.1752 - val_loss: 0.1832
Epoch 4/10
11610/11610 [==============================] - 1s 48us/sample - loss: 0.1741 - val_loss: 0.1742
Epoch 5/10
11610/11610 [==============================] - 1s 48us/sample - loss: 0.1725 - val_loss: 0.1630
Epoch 6/10
11610/11610 [==============================] - 1s 48us/sample - loss: 0.1714 - val_loss: 0.1751
Epoch 7/10
11610/11610 [==============================] - 1s 47us/sample - loss: 0.1702 - val_loss: 0.1698
Epoch 8/10
11610/11610 [==============================] - 1s 46us/sample - loss: 0.1693 - val_loss: 0.1663
Epoch 9/10
11610/11610 [==============================] - 1s 48us/sample - loss: 0.1682 - val_l

In [34]:
model.save('models\my_model_with_a_custom_loss_threshold_2.h5')

In [35]:
model = keras.models.load_model('models\my_model_with_a_custom_loss_threshold_2.h5', custom_objects={'huber_fn':create_huber(2.0)})

In [36]:
class HuberLoss(keras.losses.Loss):
    def __init__ (self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss =tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold':self.threshold}

In [39]:
model.compile(loss = HuberLoss(2.0), optimizer = 'nadam', metrics=['mae'])

In [40]:
model.fit(x_train_scaled, y_train, epochs= 10, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 1s 93us/sample - loss: 0.1799 - mae: 0.4287 - val_loss: 0.1987 - val_mae: 0.4422
Epoch 2/10
11610/11610 [==============================] - 1s 50us/sample - loss: 0.1807 - mae: 0.4272 - val_loss: 0.2046 - val_mae: 0.4418
Epoch 3/10
11610/11610 [==============================] - 1s 50us/sample - loss: 0.1787 - mae: 0.4264 - val_loss: 0.2092 - val_mae: 0.4350
Epoch 4/10
11610/11610 [==============================] - 1s 51us/sample - loss: 0.1802 - mae: 0.4271 - val_loss: 0.1800 - val_mae: 0.4189
Epoch 5/10
11610/11610 [==============================] - 1s 50us/sample - loss: 0.1777 - mae: 0.4251 - val_loss: 0.1933 - val_mae: 0.4322
Epoch 6/10
11610/11610 [==============================] - 1s 50us/sample - loss: 0.1779 - mae: 0.4245 - val_loss: 0.1947 - val_mae: 0.4306
Epoch 7/10
11610/11610 [==============================] - 1s 50us/sample - loss: 0.1774 - mae: 0.4239 - val_loss: 0.1

In [41]:
model.save("models\my_model_with_a_custom_loss_class.h5")

In [51]:
model.loss.threshold

2.0

In [52]:
keras.backend.clear_session()

## 활성화 함수, 초기화, 규제, 제한을 커스텀마이징하기

In [54]:
def my_softplus(z):   # tf.nn.softplus(z) 와 동일한 반환값
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0]+ shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):           #tf.nn.relu(weight) 와 동일한 반환값
    return tf.where(weights <0., tf.zeros_like(weights), weights)

In [55]:
layer = keras.layers.Dense(30, activation=my_softplus,
                          kernel_initializer=my_glorot_initializer,
                          kernel_constraint= my_positive_weights,
                          kernel_regularizer=my_l1_regularizer)

In [56]:
keras.backend.clear_session()

In [59]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', kernel_initializer='lecun_normal', input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                      kernel_regularizer= my_l1_regularizer,
                      kernel_initializer= my_glorot_initializer,
                      kernel_constraint= my_positive_weights),
])

In [62]:
model.compile(loss='mse', optimizer='nadam', metrics=['mae'])

In [63]:
model.fit(x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 106us/sample - loss: 1.6916 - mae: 0.9308 - val_loss: inf - val_mae: inf
Epoch 2/2
11610/11610 [==============================] - 1s 57us/sample - loss: 0.5740 - mae: 0.5113 - val_loss: 3.0938 - val_mae: 0.5218


In [64]:
model.save("./models/my_model_with_many_custom_parts.h5")

In [65]:
model = keras.models.load_model('./models/my_model_with_many_custom_parts.h5',
                               custom_objects={
                                   'my_l1_regularizer' : my_l1_regularizer,
                                   'my_softplus' : my_softplus,
                                   'my_glorot_initializer' : my_glorot_initializer,
                                   'my_positive_weights': my_positive_weights,
                                   
                               })

In [66]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor* weights))
    def get_config(self):
        return {'factor' :self.factor}

In [67]:
keras.backend.clear_session()

In [71]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', kernel_initializer='lecun_normal',
                      input_shape= input_shape),
    keras.layers.Dense(1, activation= my_softplus,
                      kernel_constraint=my_positive_weights,
                      kernel_initializer= my_glorot_initializer,
                      kernel_regularizer=MyL1Regularizer(0.01))
    
])

In [72]:
model.compile(loss='mse', optimizer = 'nadam', metrics=['mae'])

In [73]:
model.fit(x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 103us/sample - loss: 1.7441 - mae: 0.8924 - val_loss: inf - val_mae: inf
Epoch 2/2
11610/11610 [==============================] - 1s 55us/sample - loss: 0.6944 - mae: 0.5302 - val_loss: inf - val_mae: inf


In [74]:
model.save(".\models\my_model_with_many_custom_parts.h5")

In [75]:
model=keras.models.load_model(".\models\my_model_with_many_custom_parts.h5", custom_objects={
    'my_positive_weights': my_positive_weights,
    'my_glorot_initializer' : my_glorot_initializer,
    'my_softplus' : my_softplus,
    'MyL1Regularizer' : MyL1Regularizer
})

In [76]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

In [79]:
model.fit(x_train_scaled, y_train, epochs=2)

Train on 11610 samples
Epoch 1/2
11610/11610 [==============================] - 1s 77us/sample - loss: 0.5759 - huber_fn: 0.2099
Epoch 2/2
11610/11610 [==============================] - 1s 46us/sample - loss: 0.5252 - huber_fn: 0.2019


In [82]:
# 수학적으로 말하면 손실 = 지표 * 샘플 가중치의 평균(더하기 약간의 부동 소수점 정밀도 오차)입니다.
sample_weight = np.random.rand(len(y_train))
history = model.fit(x_train_scaled, y_train, epochs=2, sample_weight=sample_weight)

  ...
    to  
  ['...']
Train on 11610 samples
Epoch 1/2
11610/11610 [==============================] - 1s 52us/sample - loss: 0.2624 - huber_fn: 0.1959
Epoch 2/2
11610/11610 [==============================] - 1s 51us/sample - loss: 0.2602 - huber_fn: 0.1953


In [83]:
history.history["loss"][0], history.history["huber_fn"][0] * sample_weight.mean()

(0.2624044344150024, 0.09832823267094727)

In [87]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], 
          [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [89]:
precision([0, 1, 0, 0, 1, 0, 1, 1], 
          [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [92]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [94]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [95]:
precision.reset_states()

In [96]:
class HuberMetic(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight('total', initializer='zeros')
        self.count = self.add_weight('count', initializer='zeros')
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold':self.threshold}

In [97]:
# 위 코드를 간단하게
class HuberMetic(keras.metrics.Mean):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        super(HuberMetic, self).update_state(metric, sample_weight)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold':self.threshold}

## 사용자 정의층

In [98]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [99]:
exponential_layer([-1., 0., 1.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.36787945, 1.        , 2.7182817 ], dtype=float32)>

In [100]:
keras.backend.clear_session()

In [102]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=input_shape),
    keras.layers.Dense(1),
    exponential_layer
])
model.compile(loss='mse', optimizer='nadam')
model.fit(x_train_scaled, y_train, epochs=5, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 1s 94us/sample - loss: 1232675.2384 - val_loss: 45792190760119538822372392960.0000
Epoch 2/5
11610/11610 [==============================] - 1s 51us/sample - loss: 243.6863 - val_loss: 44181284297741804851838320640.0000
Epoch 3/5
11610/11610 [==============================] - 1s 52us/sample - loss: 241.6042 - val_loss: 41812132064087386519291035648.0000
Epoch 4/5
11610/11610 [==============================] - 1s 51us/sample - loss: 238.9172 - val_loss: 39215979173277675115872518144.0000
Epoch 5/5
11610/11610 [==============================] - 1s 49us/sample - loss: 235.9405 - val_loss: 36019566810859288001964408832.0000


In [103]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name ='kernel', shape=[batch_input_shape[-1], self.units],
                                     initializer='glorot_normal')
        self.bias = self.add_weight(name='bias', shape=[self.units], initializer='zeros')
        super().build(batch_input_shape)
        
    def call(self, x):
        return self.activation(x @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units':self.units, 'activation':keras.activations.serialize(self.activation)}

In [104]:
keras.backend.clear_session()

In [105]:
model = keras.models.Sequential([
    MyDense(30, activation='relu', input_shape=input_shape),
    MyDense(1)
])

In [106]:
model.compile(loss='mse', optimizer='nadam')

In [107]:
model.fit(x_train_scaled, y_train, epochs = 2, validation_data=(x_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 84us/sample - loss: 1.2291 - val_loss: 0.7103
Epoch 2/2
11610/11610 [==============================] - 1s 49us/sample - loss: 0.5603 - val_loss: 0.6966


In [108]:
model.evaluate(x_test_scaled, y_test)

5160/5160 [==============================] - 0s 24us/sample - loss: 0.4739


0.4739317478597626

In [109]:
model.save(".\models\my_model_with_a_custom_layer.h5")

In [110]:
model = keras.models.load_model(".\models\my_model_with_a_custom_layer.h5", custom_objects={'MyDense': MyDense})

In [111]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, x):
        x1, x2 = x
        return x1+ x2, x1* x2
    
    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

In [112]:
keras.backend.clear_session()

In [113]:
inputs1 = keras.layers.Input(shape=[2])
inputs2 = keras.layers.Input(shape=[2])
outputs1, outputs2 = MyMultiLayer()((inputs1, inputs2))

In [114]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [118]:
model.compile(loss="mse", optimizer="nadam")
model.fit(x_train_scaled, y_train, epochs=2,
          validation_data=(x_valid_scaled, y_valid), verbose=2)
model.evaluate(x_test_scaled, y_test, verbose =2)

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 - 1s - loss: 0.3841 - val_loss: 0.4985
Epoch 2/2
11610/11610 - 1s - loss: 0.3704 - val_loss: 0.3530
5160/5160 - 0s - loss: 0.3727


0.3726829981388048